In [9]:
# ライブラリのインポート
import pandas as pd
import numpy as np
from itertools import product

In [10]:
# 入力ファイル名
input_file = R'C:/Users/sukegawa/Desktop/study/datasets/invfp/invfp_8.csv'
# 出力ファイル名
output_file = R'C:/Users/sukegawa/Desktop/study/datasets/tinvfp/3tinvfp_8.csv'

In [11]:
# データの読み込み
data = pd.read_csv(input_file)

In [12]:
# 閾値
threshold = 20

# チャンクサイズ
chunk_size = 100

# 変位と距離の計算を行う関数
def calc_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    distance = np.sqrt(dx**2 + dy**2)
    return distance, dx, dy

# 3時点の協調位置指紋を作成する関数
def make_3tinvfp(data1, data2, data3):
    results = []

    # 1→2点間,2→3点間の変位と距離を計算
    for row1, row2, row3 in product(data1.values, data2.values, data3.values):
        dist12, dx12, dy12 = calc_distance(row1[0], row1[1], row2[0], row2[1])
        if dist12 > threshold:
            continue
        
        dist23, dx23, dy23 = calc_distance(row2[0], row2[1], row3[0], row3[1])
        if dist23 > threshold:
            continue
        
        results.append({
            'x': row1[0], 'y': row1[1],
            'x_2': row2[0], 'y_2': row2[1],
            'x_3': row3[0], 'y_3': row3[1],
            'dx_2': dx12, 'dy_2': dy12, 'distance_2': dist12,
            'dx_3': dx23, 'dy_3': dy23, 'distance_3': dist23
        })
    
    return pd.DataFrame(results)

# 結果をcsvファイルに書き込みする関数
def chunked_processing(input_file, output_file, chunk_size):
    chunks = []
    for chunk in pd.read_csv(input_file, chunksize=chunk_size):
        chunks.append(chunk[['x', 'y']])
    
    total_chunks = len(chunks)
    
    
    first_chunk = True
    for i in range(total_chunks):
        for j in range(i, total_chunks):
            for k in range(j, total_chunks):
                result = make_3tinvfp(chunks[i], chunks[j], chunks[k])
                if not result.empty:
                    result.to_csv(output_file, mode='a', header=first_chunk, index=False)
                    first_chunk = False
                print(f"Processed chunks {i}, {j}, {k}")

# 書き込み
chunked_processing(input_file, output_file, chunk_size)


Processed chunks 0, 0, 0
Processed chunks 0, 0, 1
Processed chunks 0, 1, 1
Processed chunks 1, 1, 1
